# Esercizio 7

Filtrare da un file in formato `FASTQ` i reads che hanno una percentuale di basi con qualità almeno Q che risulta essere almeno P. Per ognuno di essi effettuare il *trimming* (ottenendo la più lungha sottostringa in cui tutte le basi hanno qualità almeno Q) e produrlo in output (in formato `FASTQ`) solo se la porzione rimasta è lunga almeno L basi.

***

Parametri in input:

- nome del file in formato `FASTQ`
- soglia minima di qualità Q
- percentuale minima P (numero compreso tra 0 e 1) delle basi con qualità almeno Q
- lunghezza minima L per tenere un read dopo il *trimming*

***

Requisiti:
- deve essere definita la funzione `ascii_to_quality()` che prende come argomento un carattere ASCII e restituisce il corrispondente Phred Value q.
- deve essere definita la funzione `get_quality_percentage()` che prende in input una stringa di qualità (cioé una stringa di caratteri ASCII che codificano Phred Values) e una soglia minima, e restituisce la percentuale di caratteri che codificano una qualità pari ad almeno la soglia minima.
- deve essere definita la funzione `get_trimming_interval()` che prende in input una stringa di qualità e una soglia minima, e restituisce il più lungo intervallo di posizioni contenente solo caratteri che codificano qualità almeno pari alla soglia minima.
- deve essere definita la funzione `get_trimmed_read()` che prende in input una soglia minima di qualità e un read in  formato `FASTQ` inteso come lista dei suoi quattro *record* `FASTQ`, ed effettua il *trimming*, cioé trova la più lunga sottostringa del read le cui basi hanno una qualità pari ad almeno la soglia passata come argomento. La funzione deve restituire la lista dei quattro *record* `FASTQ` del *trimmed* read (NB: anche la stringa di qualità deve quindi essere opportunamente sottoposta a *trimming* prima di produrre in output il read). Tale funzione dovrà invocare la precedente `get_trimming_interval()`


**Esempio:** Q=58.

Read prima del *trimming*:

        @HWUSI-EAS522:8:5:662:692#0/1
        TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC
        +HWUSI-EAS522:8:5:662:692#0/1
        aaaa`aa`aa`]__`aa`_U[_a`^\\UTWZ`X^QX
        
Read dopo il *trimming*:

        @HWUSI-EAS522:8:5:662:692#0/1
        TATGGAGGCCCAACTTCTT
        +HWUSI-EAS522:8:5:662:692#0/1,
        aaaa`aa`aa`]__`aa`_
        
in cui viene tenuto un prefisso.

***

Variabili di output:
- `output_list`, lista dei reads prodotti in formato `FASTQ`. Ogni elemento (read) della lista è una lista annidata contenente i quattro *record* del formato `FASTQ`.

***

## Soluzione

### Parametri in input

In [60]:
fastq_file_name = './input.fq'

Soglia minima di qualità Q:

In [61]:
quality_threshold = 58

Minima percentuale P di basi aventi qualità minima Q:

In [62]:
min_percentage = 0.7

Lunghezza minima L

In [63]:
min_length = 20

### Definizione della funzione `ascii_to_quality()`

L'argomento è un carattere ASCII che codifica un Phred Value. La funzione restituisce la qualità codificata dal carattere.

### Definizione della funzione `get_quality_percentage()`

Il primo argomento della funzione è una stringa di qualità, mentre il secondo argomento è una soglia di qualità. La funzione restituisce la percentuale (numero compreso tra 0 e 1) di caratteri che codificando una qualità almeno pari alla soglia.

**NOTA BENE**: supporre che il carattere `\n` sia già stato rimosso dalla fine della stringa `quality_string` passata come argomento.

### Definizione della funzione `get_trimming_interval()`

Il primo argomento della funzione è la stringa di qualità, mentre il secondo argomento è soglia di qualità. La funzione restituisce le posizioni di inizio e fine del *trimming interval* sulla stringa di qualità, cioé della più lunga sottostringa composta solo di caratteri che codificano una qualità almeno pari alla soglia.

**NOTA BENE**: supporre che il carattere `\n` sia già stato rimosso dalla fine della stringa `quality_string` passata come argomento.

#### Spiegazione della funzione `get_trimming_interval()`

Si consideri la seguente stringa di qualità (primo argomento della funzione):

In [67]:
quality_string = 'aaaa`aa`aa`]__`aa`_U[_a`^\\\\UTWZ`X^QX'

La soglia di qualità minima (secondo argomento della funzione) è quella scelta sopra.

In [68]:
quality_threshold

58

Si costruisca la lista `bool_list` di valori di tipo `bool` lunga come la stringa di qualità, tale per cui il valore i-esimo è `True` se il carattere i-esimo della stringa di qualità codifica un Phred Value almeno pari alla soglia `quality_threshold`.

In [70]:
bool_list

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False]

Basta a questo punto trovare e restituire il più lungo intervallo di posizioni nella lista `bool_list` che contenga solo valori `True`.
    
Costruire quindi la lista `start_list` di tutte le posizioni di inizio di un intervallo di valori `True` lungo `bool_list`, elencate da sinistra a destra lungo `bool_list`.

Una posizione `i>0` è di inizio di un intervallo di valori `True` (anche di ampiezza 1) se `bool_list[i]` è uguale a `True` e `bool_list[i-1]` è uguale a `False`.

In particolare, la posizione `i=0` è una posizione di inizio di un intervallo di valori `True` se `bool_list[i]` è uguale a `True`.

In [72]:
start_list

[0, 20, 31, 33]

Aggiungere all'inizio di `start_list` una *fake position* pari a 1, per gestire il caso in cui tutti i valori in `bool_list` siano uguali a `False` (tutti i caratteri della stringa di qualità codificano valori di qualità al di sotto della soglia minima).

In [74]:
start_list

[1, 0, 20, 31, 33]

Costruire in modo analogo la lista `end_list` di tutte le posizioni di fine di un intervallo di valori `True` lungo `bool_list`, elencate da sinistra a destra lungo `bool_list`.

Una posizione `j < len(bool_list)-1` è di fine di un intervallo di valori `True` (anche di ampiezza 1) se `bool_list[j]` è uguale a `True` e `bool_list[j+1]` è uguale a `False`.

In particolare, la posizione `j = len(bool_list)-1` è una posizione di fine di un intervallo di valori `True` se `bool_list[len(bool_list)-1]` è uguale a `True`.

In [76]:
end_list

[18, 26, 31, 33]

Aggiungere all'inizio di `end_list` una *fake position* pari a 0, per gestire il caso in cui tutti i valori in `bool_list` siano uguali a `False` (tutti i caratteri della stringa di qualità codificano valori di qualità al di sotto della soglia minima).

In [78]:
end_list

[0, 18, 26, 31, 33]

`start_list[p]` ed `end_list[p]` (per una data posizione p) sono le due posizioni di inizio e di fine sulla lista `bool_list` di un intervallo di soli valori `True`. Quindi `end_list[p]-start_list[p]+1` è la lunghezza del p-esimo intervallo a partire da sinistra.

Per `p=0` si ha `start_list[0]=1` e `end_list[0]=0`, che sono le posizioni di inizio e fine del *fake interval* che ha lunghezza 0.

A questo punto basta determinare l'intervallo più lungo.

Costruire quindi la lista `interval_lengths` delle lunghezza degli intervalli.

In [80]:
interval_lengths

[0, 19, 7, 1, 1]

Estrarre l'inizio e la fine dell'intervallo di lunghezza massima (che sarà il *trimming interval*).

In [82]:
start_interval

0

In [83]:
end_interval

18

Si faccia in modo che la funzione `get_trimming_interval()` restituisca la lista dei due elementi `[start_interval, end_interval+1]` che rappresenta il *trimming interval*.

Il fatto di sommare 1 a `end_interval` prima di restituirlo è per rendere l'intervallo subito pronto per effettuare l'operazione di *slicing* (trimming) della stringa di qualità e del relativo read.

**NOTA BENE**: se viene restituito il *fake interval* `[1,1]` allora significa che tutti i valori in `bool_list` sono uguali a `False` (cioé nella stringa di qualità tutti i valori sono al di sotto della soglia minima), e il trimming restituirà una stringa vuota.

### Definizione della funzione `get_trimmed_read()`

Il primo argomento della funzione è una soglia minima di qualità, mentre il secondo argomento è una lista dei quattro *record* di un read in formato `FASTQ`. I *record* prodotti in output devono avere il il carattere `\n` alla fine.

**NOTA BENE:** supporre che il carattere `\n` non sia ancora stato tolto dalla fine di ognuno dei quattro *record*.

---

### Lettura del file in formato `FASTQ`

Lettura del file `FASTQ` in input nella lista `fastq_file_rows`

In [86]:
fastq_file_rows

['@HWUSI-EAS522:8:5:662:692#0/1\n',
 'TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC\n',
 '+HWUSI-EAS522:8:5:662:692#0/1\n',
 'aaaa`aa`aa`]__`aa`_U[_a`^\\\\UTWZ`X^QX\n',
 '@HWUSI-EAS522:8:5:662:693#0/1\n',
 'TCTGCCAACTTCTTATGGAGGCCTGTATTCACAGGT\n',
 '+HWUSI-EAS522:8:5:662:693#0/1\n',
 'Aaaa`aa`aa`]__`:a`_U;_A`^\\\\UTWZ`X^QX\n',
 '@HWUSI-EAS522:8:5:662:694#0/1\n',
 'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG\n',
 '+HWUSI-EAS522:8:5:662:694#0/1\n',
 'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^QX\n',
 '@HWUSI-EAS522:8:5:662:695#0/1\n',
 'TCGCCTGTATTCACAGGTTGCCAACTTCTTATGGAG\n',
 '+HWUSI-EAS522:8:5:662:695#0/1\n',
 'AaaA`aa`aa`]__`:A`_U;_A`^\\\\UTWZ`X^QX\n']

Costruire la lista `read_header_pos` degli indici 0-based di posizione dei *record* di intestazione dei read (quelli che iniziano con il carattere `@`) all'interno della lista `fastq_file_rows`.

Usare la lista `read_header_pos` per raggruppare i *record* della lista `fastq_file_rows` per read (cioé gruppi di quattro *record* ciascuno), e inserire tali raggruppamenti nella lista `output_list`.

In [90]:
output_list

[['@HWUSI-EAS522:8:5:662:692#0/1\n',
  'TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC\n',
  '+HWUSI-EAS522:8:5:662:692#0/1\n',
  'aaaa`aa`aa`]__`aa`_U[_a`^\\\\UTWZ`X^QX\n'],
 ['@HWUSI-EAS522:8:5:662:693#0/1\n',
  'TCTGCCAACTTCTTATGGAGGCCTGTATTCACAGGT\n',
  '+HWUSI-EAS522:8:5:662:693#0/1\n',
  'Aaaa`aa`aa`]__`:a`_U;_A`^\\\\UTWZ`X^QX\n'],
 ['@HWUSI-EAS522:8:5:662:694#0/1\n',
  'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG\n',
  '+HWUSI-EAS522:8:5:662:694#0/1\n',
  'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^QX\n'],
 ['@HWUSI-EAS522:8:5:662:695#0/1\n',
  'TCGCCTGTATTCACAGGTTGCCAACTTCTTATGGAG\n',
  '+HWUSI-EAS522:8:5:662:695#0/1\n',
  'AaaA`aa`aa`]__`:A`_U;_A`^\\\\UTWZ`X^QX\n']]

Eliminare da `output_list` i read la cui percentuale di basi con qualità almeno pari alla soglia minima (parametro `quality_threshold`) è inferiore al parametro `min_percentage`.

In [92]:
output_list

[['@HWUSI-EAS522:8:5:662:692#0/1\n',
  'TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC\n',
  '+HWUSI-EAS522:8:5:662:692#0/1\n',
  'aaaa`aa`aa`]__`aa`_U[_a`^\\\\UTWZ`X^QX\n'],
 ['@HWUSI-EAS522:8:5:662:694#0/1\n',
  'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG\n',
  '+HWUSI-EAS522:8:5:662:694#0/1\n',
  'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^QX\n']]

Effettuare il *trimming* dei reads rimasti in `output_list`.

In [97]:
output_list

[['@HWUSI-EAS522:8:5:662:692#0/1\n',
  'TATGGAGGCCCAACTTCTT\n',
  '+HWUSI-EAS522:8:5:662:692#0/1\n',
  'aaaa`aa`aa`]__`aa`_\n'],
 ['@HWUSI-EAS522:8:5:662:694#0/1\n',
  'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTA\n',
  '+HWUSI-EAS522:8:5:662:694#0/1\n',
  'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^\n']]

Eliminare da `output_list` i reads troppo corti.

In [99]:
output_list

[['@HWUSI-EAS522:8:5:662:694#0/1\n',
  'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTA\n',
  '+HWUSI-EAS522:8:5:662:694#0/1\n',
  'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^\n']]